In [ ]:
#install all dependences
pip install boto3 pandas openpyxl xlrd gspread oauth2client

In [427]:
#all imports
import pandas as pd
import boto3
import os
from dotenv import load_dotenv
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from openpyxl import load_workbook

In [428]:
#loading the function to load all variable
load_dotenv()

True

In [429]:
#get all access key to get the xlsx file on aws S3
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("BUCKET_NAME")

In [430]:
#initialise the connection on aws S3
s3 = boto3.client(
    "s3",
    aws_access_key_id = access_key ,
    aws_secret_access_key = secret_key 
)


In [431]:
#function to download the file from aws s3
def download_xl_file(bucket_name , s3_key , local_file) :
    try :
        s3.download_file(bucket_name , s3_key , local_file)
        print ("download success")
    except Exception as e :
        print (f"error : '{e}'")

In [432]:
#function to apload a file on aws s3
def upload_csv_file(bucket_name , s3_key , local_file) :
    try :
        s3.upload_file(local_file , bucket_name , s3_key)
        print ("upload success")
    except Exception as e :
        print (f"error : '{e}'")

In [433]:
#empty all xl file rows
def empty_xl_file(xl_file) :
    wb = load_workbook(xl_file)
    ws = wb.active
    if ws.max_row > 1 :
        ws.delete_rows(2 , ws.max_row - 1)
    wb.save(xl_file)

In [434]:
#calling the function to download file from aws to load the data
s3_key = "sondagepublic/sondage.xlsx"
local_file = 'sondage.xlsx'
download_xl_file(BUCKET_NAME , s3_key , local_file)

download success


In [435]:
#transform the xl file to data frame
data = pd.read_excel(local_file , engine = "openpyxl")
print(data)

     product  quantity        date  price_before  price_after      prom_type  \
0  chaussure        12  2025-04-15           120          200  offre grouper   

  gender  age  income  
0  homme   22    1000  


### these are the following step for cleaning
* delete a row or a column wich is totaly empty
* rename all columns name : replace the space by '_' , convert all to lower case
* delete all duplicate data
* reformat all columns data 

In [436]:

def delete_empty_row_col(df) :
    df.dropna(how = 'all' , inplace = True)
    df.dropna(axis=1 , how = 'all' , inplace = True)


In [437]:
def rename_col(df) :
    df.columns = df.columns.str.strip().str.lower().str.replace(' ','_')

In [438]:
def delete_duplicate(df) :
    df.drop_duplicates(inplace=True)

In [439]:
print(data.product)

<bound method DataFrame.prod of      product  quantity        date  price_before  price_after      prom_type  \
0  chaussure        12  2025-04-15           120          200  offre grouper   

  gender  age  income  
0  homme   22    1000  >


In [440]:
print(data.columns)

Index(['product', 'quantity', 'date', 'price_before', 'price_after',
       'prom_type', 'gender', 'age', 'income'],
      dtype='object')


In [441]:
def replace_product(df) :
    if len(df) != 0 :
        df["product"] = df["product"].fillna("unknown")
    return df

In [442]:
print(data.columns)

Index(['product', 'quantity', 'date', 'price_before', 'price_after',
       'prom_type', 'gender', 'age', 'income'],
      dtype='object')


In [443]:
def replace_qty(df) :
    df["quantity"] = df["quantity"].fillna(1) 
    return df

### prices ajustment
there are two prices columns : before the promotion and after the promotion , normaly the price before should be higher than the price during the promotion there could be a possibility that the customer have reverse the two value so this function ajust it by reversing the values on right way

In [444]:

def ajust_prices(df) :
    df.dropna(subset=["price_before"] , inplace=True)
    df.dropna(subset=["price_after"] , inplace = True)
    before = df["price_before"]
    after = df["price_after"]
    mask = df["price_after"] > df["price_before"]
    temp = df.loc[mask, "price_before"].copy()
    df.loc[mask, "price_before"] = df.loc[mask, "price_after"]
    df.loc[mask, "price_after"] = temp

In [445]:
def put_lower_case(df) :
    df["product"] = df["product"].astype(str).str.lower()
    df["prom_type"] = df["prom_type"].astype(str).str.lower()
    df["gender"] = df["gender"].astype(str).str.lower()

In [446]:
print(data.columns)

Index(['product', 'quantity', 'date', 'price_before', 'price_after',
       'prom_type', 'gender', 'age', 'income'],
      dtype='object')


In [447]:
def delete_promo_type(df) :
    df.drop(df[df["prom_type"].isna() | (df["prom_type"] == "")].index, inplace=True)

In [448]:
print(data.prom_type)

0    offre grouper
Name: prom_type, dtype: object


In [449]:
def write_xl_file(file , df) :
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name('sheet_secret.json', scope)
    client = gspread.authorize(creds)
    spreadsheet = client.open(file) 
    worksheet = spreadsheet.sheet1
    for row in df.values.tolist():
        worksheet.append_row(row)

    print("Les données ont été téléchargées dans Google Sheets.")

In [450]:
def delete_sondage_data() :
    path = "sondage.xlsx"
    empty_xl_file('sondage.xlsx')
    upload_csv_file(BUCKET_NAME , "sondagepublic/sondage.xlsx" , path)

after checking if there is a new data call the functons for cleaning prossess and then empty the file 

In [451]:
if len(data) != 0 :
    put_lower_case(data)
    delete_promo_type(data)
    delete_empty_row_col(data)
    rename_col(data) 
    delete_duplicate(data)
    replace_product(data)
    replace_qty(data)
    ajust_prices(data)
    data.to_csv('clean_data.csv' , index=False)
    write_xl_file("analyse" , data)
    delete_sondage_data()

Les données ont été téléchargées dans Google Sheets.
upload success


In [452]:
print(data)

     product  quantity        date  price_before  price_after      prom_type  \
0  chaussure        12  2025-04-15           200          120  offre grouper   

  gender  age  income  
0  homme   22    1000  
